# Evaluating the results of experiments of Neurips 2023, HAIR

In [2]:
%load_ext autoreload

In [3]:
%autoreload
import numpy as np
import pandas as pd
import sys
import os
import glob
import imageio
import time
import matplotlib.pyplot as plt
from scipy import ndimage, signal
from cv2 import medianBlur
import gdal
from osgeo import ogr
from osgeo import osr
import tensorflow as tf
sys.path.append("..")
sys.path.append("/home/saeid/phd/segmentation/costum_arild/source")
from PIL import Image 
from costum_arild.source.utils import image_processing_utils, gdal_utils, notebook_utils, model_utils
from costum_arild.source.data_processing import TrainingImage, divide_image, reassemble_big_image, reassemble_big_image_np

2023-06-07 14:56:03.656595: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Segmentation Models: using `tf.keras` framework.


In [4]:
def read_arr(file_path):
    with open(file_path) as f:
        lines = f.read().splitlines()[0]
        lines = lines.replace('  ]', ']').replace(' ]', ']')
        iou = lines.strip('][').replace('   ',' ').replace('  ',' ').split(' ')
        print(iou)
        iou = np.array([round(float(numbers), 4) for numbers in iou])
        return iou

def read_miou_file(file_path):
    with open(file_path) as f:
        lines = f.read().splitlines()[0]
        return round(float(lines), 4)

In [5]:
def list_iou_model(exp_path, transformer=False):
    iou_list = []
    miou_list = []
    list_direct = glob.glob( os.path.join(exp_path, '*'))
    list_direct.sort()

    for idx, _ in enumerate(list_direct):
                            
        model_name = os.path.split(list_direct[idx])[-1]
        # read one csv file of each IOU
        base_path = os.path.join(exp_path, model_name)
        if transformer:
            arr_path = os.path.join(base_path, f'checkpoints_0_arr.txt')
            miou_path = os.path.join(base_path, f'checkpoints_0_overall.txt')
        else:
            arr_path = os.path.join(base_path, f'{model_name}_0_arr.txt')
            miou_path = os.path.join(base_path, f'{model_name}_0_overall.txt')
        all_ious = read_arr(file_path=arr_path)
        miou = read_miou_file(file_path=miou_path)
        iou_list.append(all_ious)
        miou_list.append(miou)
        
    iou_np = np.stack(iou_list, axis=0)
    miou_np = np.stack(miou_list, axis=0)
    return iou_np, miou_np

# IOD

## UNet

In [ ]:
### normal

In [129]:
exp_type = 'IOD'
model_arch = 'unet'
pretrain_type = 'normal'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.46588535', '0.59706813', '0.88396026', '0.85407462', '0.65853684', '0.84232637']
['0.46771946', '0.60981968', '0.87735712', '0.84606875', '0.63062861', '0.85182533']
['0.46664613', '0.61475258', '0.88044651', '0.85520761', '0.66451647', '0.85460607']
['0.46573718', '0.6073332', '0.87907312', '0.85430058', '0.66578056', '0.86424558']
['0.46763112', '0.61133848', '0.88016909', '0.85583393', '0.66551516', '0.86394231']


(array([0.00083187, 0.0059942 , 0.00216924, 0.00355359, 0.01345472,
        0.00818403]),
 array([0.4667 , 0.60806, 0.88022, 0.8531 , 0.65698, 0.85536]),
 0.004772672207474553,
 0.7707399999999999,
 array([0.7672, 0.7631, 0.7739, 0.7741, 0.7754]))

### pretrain

In [130]:
exp_type = 'IOD'
model_arch = 'unet'
pretrain_type = 'pretrain'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.46827507', '0.57302558', '0.88232005', '0.85126974', '0.65060758', '0.85660422']
['0.46698536', '0.61097381', '0.87493081', '0.85173442', '0.65065595', '0.84962538']
['0.46524901', '0.59701316', '0.87773917', '0.85480567', '0.66311281', '0.84653964']
['0.46732851', '0.55931913', '0.87871271', '0.84419365', '0.6213975', '0.83919763']
['0.46749459', '0.56940163', '0.87907156', '0.84845445', '0.65120226', '0.84173895']


(array([0.00102489, 0.01908419, 0.00238462, 0.00356258, 0.0138424 ,
        0.00612712]),
 array([0.46706, 0.58194, 0.87854, 0.8501 , 0.6474 , 0.84672]),
 0.007152510048926864,
 0.7609600000000001,
 array([0.7628, 0.7676, 0.7678, 0.7486, 0.758 ]))

## HRNet

### normal

In [140]:
exp_type = 'IOD'
model_arch = 'hrnet'
pretrain_type = 'normal_v1'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.46594815', '0.54720675', '0.87096733', '0.83120487', '0.62456684', '0.76485864']
['0.46457324', '0.45984974', '0.86868755', '0.81223039', '0.62735499', '0.76982578']
['0.46705218', '0.54728537', '0.87664935', '0.82790524', '0.6405874', '0.7924218']
['0.46615998', '0.53128469', '0.87165026', '0.81984282', '0.62421353', '0.77662546']
['0.46607538', '0.54228648', '0.87031761', '0.81894599', '0.63157534', '0.78118124']


(array([0.00080349, 0.033402  , 0.00266278, 0.00678145, 0.00606577,
        0.00952059]),
 array([0.46598, 0.52558, 0.87166, 0.822  , 0.62968, 0.77698]),
 0.009692264957170738,
 0.7252,
 array([0.7278, 0.7076, 0.737 , 0.7247, 0.7289]))

### pretrain

In [144]:
exp_type = 'IOD'
model_arch = 'hrnet'
pretrain_type = 'pretrain_v1'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.449981', '0.55174221', '0.86629826', '0.83099929', '0.63555813', '0.75492796']
['0.46663363', '0.56921579', '0.87827113', '0.83296181', '0.64618892', '0.80921305']
['0.46600594', '0.55044029', '0.8594971', '0.79367442', '0.64159129', '0.78018184']
['0.46606088', '0.57843927', '0.87428885', '0.82528146', '0.64261377', '0.76340374']
['0.46407408', '0.55417152', '0.87363532', '0.82441084', '0.65117536', '0.78584118']


(array([0.00633738, 0.01108917, 0.00668551, 0.01427016, 0.00516511,
        0.01888619]),
 array([0.46256, 0.56078, 0.8704 , 0.82148, 0.64344, 0.7787 ]),
 0.007852795680520414,
 0.73496,
 array([0.7279, 0.7472, 0.7251, 0.7368, 0.7378]))

## SwinTransformer

### normal

In [131]:
exp_type = 'IOD'
model_arch = 'swin'
pretrain_type = 'normal'
transformer_flag = True

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path, transformer=transformer_flag)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.46391279', '0.45316886', '0.83728316', '0.77041917', '0.47538027', '0.59727028']
['0.46359933', '0.44267962', '0.83934163', '0.77029666', '0.47891605', '0.59700063']
['0.46287669', '0.44129226', '0.83426997', '0.76107552', '0.47468569', '0.60440819']
['0.46466848', '0.40418563', '0.83664044', '0.76544486', '0.47000769', '0.61777283']
['0.46601299', '0.45148132', '0.82769991', '0.7488961', '0.45259233', '0.60079671']


(array([0.00106094, 0.01781588, 0.0040028 , 0.00794969, 0.00930277,
        0.00765862]),
 array([0.46422, 0.43858, 0.83504, 0.76322, 0.47032, 0.60346]),
 0.003973411632338159,
 0.6221,
 array([0.6267, 0.6256, 0.6231, 0.6188, 0.6163]))

### pretrain

In [132]:
exp_type = 'IOD'
model_arch = 'swin'
pretrain_type = 'pretrain'
transformer_flag = True

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path, transformer=transformer_flag)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.43694205', '0.45350378', '0.82143029', '0.74723106', '0.49630815', '0.59628621']
['0.46694163', '0.4258594', '0.82713491', '0.76389373', '0.46413108', '0.60447127']
['0.46697247', '0.42114558', '0.83205103', '0.75635428', '0.46809844', '0.60202204']
['0.46413376', '0.4686201', '0.84653695', '0.77831228', '0.52478927', '0.65341522']
['0.4650646', '0.45120176', '0.84423883', '0.76951508', '0.53607809', '0.61259591']


(array([0.01160207, 0.01788436, 0.00969466, 0.01068019, 0.02903752,
        0.02050157]),
 array([0.46   , 0.44406, 0.83426, 0.76306, 0.49788, 0.61376]),
 0.015259095648169988,
 0.6306,
 array([0.623 , 0.6171, 0.6159, 0.6543, 0.6427]))

## FPN

### normal

In [1]:
exp_type = 'IOD'
model_arch = 'fpn'
pretrain_type = 'normal'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

NameError: name 'list_iou_model' is not defined

### pretrain

In [66]:
exp_type = 'IOD'
model_arch = 'fpn'
pretrain_type = 'pretrain'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.44768605', '0.6039146', '0.8719792', '0.85058165', '0.64357687', '0.83852743']
['0.46602461', '0.59442514', '0.87690817', '0.85286297', '0.6553677', '0.84611792']
['0.46638341', '0.60177611', '0.87548375', '0.84763396', '0.65363734', '0.84515542']
['0.46512451', '0.59739696', '0.8733741', '0.85066154', '0.64771246', '0.84026819']
['0.46768489', '0.57267048', '0.87216489', '0.83557458', '0.6347341', '0.83248522']


(array([0.0074869 , 0.01117365, 0.00191102, 0.00617459, 0.0074493 ,
        0.00492926]),
 array([0.46258, 0.59404, 0.874  , 0.84748, 0.647  , 0.84052]),
 0.005712582603341493,
 0.76058,
 array([0.7617, 0.7651, 0.7647, 0.7619, 0.7495]))

## DeepLab

### normal

In [7]:
exp_type = 'IOD'
model_arch = 'deeplab'
pretrain_type = 'normal'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.46024322', '0.63166248', '0.88580913', '0.86993277', '0.67642134', '0.88038937']
['0.4550538', '0.61807068', '0.88079142', '0.86693059', '0.66915726', '0.87993088']
['0.45432734', '0.60219432', '0.87786642', '0.85582737', '0.64158265', '0.85003373']
['0.45942892', '0.60556136', '0.87784211', '0.86324635', '0.64651974', '0.84769755']
['0.45640777', '0.59586727', '0.87368995', '0.8557105', '0.63771842', '0.85165356']


(array([0.00233358, 0.01275194, 0.0040005 , 0.00575396, 0.01554418,
        0.01492335]),
 array([0.45708, 0.6107 , 0.8792 , 0.8623 , 0.65428, 0.86194]),
 0.010282295463562591,
 0.77368,
 array([0.7888, 0.783 , 0.7655, 0.7682, 0.7629]))

### pretrain

In [ ]:
exp_type = 'IOD'
model_arch = 'deeplab'
pretrain_type = 'pretrain'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

mious.std(axis=0), mious.mean(axis=0)

## MagNet

### normal

In [5]:
exp_type = 'IOD'
model_arch = 'magnet'
pretrain_type = 'normal'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.21417168', '0.55251818', '0.77489687', '0.63880091', '0.57148651', '0.6702454']
['0.3918772', '0.46799825', '0.77784278', '0.80458978', '0.58365602', '0.56059259']
['0.4551821', '0.51363152', '0.73474769', '0.78551189', '0.588673', '0.59736654']
['0.39253724', '0.52170298', '0.80322759', '0.81961645', '0.56224282', '0.60077975']
['0.48562776', '0.5317847', '0.78722942', '0.7789223', '0.58125964', '0.6965154']


(array([0.09410997, 0.02797352, 0.0226912 , 0.06494316, 0.00947004,
        0.05029911]),
 array([0.38788, 0.51752, 0.77556, 0.76548, 0.57748, 0.6251 ]),
 0.013909622568567426,
 0.65222,
 array([0.6416, 0.6389, 0.644 , 0.6615, 0.6751]))

### pretrain

In [6]:
exp_type = 'IOD'
model_arch = 'magnet'
pretrain_type = 'pretrain'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.43703138', '0.41334094', '0.73713974', '0.69115444', '0.39859941', '0.52977038']
['0.1803459', '0.43465641', '0.73770653', '0.74909864', '0.47496826', '0.45518875']
['0.47507804', '0.44675607', '0.7331581', '0.71626256', '0.45639852', '0.51156147']
['0.49414384', '0.46876151', '0.68524337', '0.65391268', '0.46154598', '0.45995346']
['0.47507804', '0.44675607', '0.7331581', '0.71626256', '0.45639852', '0.51156147']


(array([0.11748447, 0.01811777, 0.02012873, 0.03162655, 0.02638275,
        0.0302058 ]),
 array([0.41232, 0.44208, 0.72528, 0.70536, 0.44958, 0.49364]),
 0.011123416741271513,
 0.5631600000000001,
 array([0.554 , 0.5703, 0.5728, 0.5459, 0.5728]))

# OOD

## UNet

### normal

In [127]:
exp_type = 'OOD'
model_arch = 'unet'
pretrain_type = 'normal'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.63798428', '0.29044107', '0.79174419', '0.74233638', '0.61123266', '0.73965764']
['0.62658365', '0.32989085', '0.7581466', '0.75228792', '0.60623248', '0.7903479']
['0.63683198', '0.31629525', '0.79812594', '0.75396925', '0.6132914', '0.721848']
['0.62975525', '0.31084762', '0.78783516', '0.76455863', '0.61687271', '0.6782845']
['0.63780784', '0.35732037', '0.78343029', '0.7859391', '0.61238353', '0.729691']


(array([0.00470064, 0.02217824, 0.01373716, 0.01483501, 0.00346814,
        0.03591989]),
 array([0.6338 , 0.32094, 0.78382, 0.75982, 0.612  , 0.73196]),
 0.0080430342533151,
 0.64174,
 array([0.6351, 0.6474, 0.6407, 0.6317, 0.6538]))

### pretrain

In [128]:
exp_type = 'OOD'
model_arch = 'unet'
pretrain_type = 'pretrain'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.62702577', '0.28684646', '0.73210007', '0.6624797', '0.54472427', '0.79014067']
['0.631227', '0.31665704', '0.75780507', '0.69816851', '0.536337', '0.77237265']
['0.63134852', '0.31717741', '0.7679255', '0.69346246', '0.53943968', '0.70826489']
['0.62564067', '0.28595702', '0.79335688', '0.61532579', '0.46551467', '0.77024394']
['0.63154241', '0.30283151', '0.77310372', '0.64823087', '0.49306539', '0.7522321']


(array([0.00250711, 0.01367304, 0.02007352, 0.03053657, 0.03116986,
        0.02788653]),
 array([0.62932, 0.3019 , 0.76486, 0.66354, 0.5158 , 0.75864]),
 0.010298427064362787,
 0.60098,
 array([0.6033, 0.6163, 0.6053, 0.5861, 0.5939]))

## HRNet

### normal

In [146]:
exp_type = 'OOD'
model_arch = 'hrnet'
pretrain_type = 'normal_v1'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.61977827', '0.24705649', '0.68353944', '0.64054177', '0.50109959', '0.6982349']
['0.62037373', '0.16780044', '0.55280133', '0.59380182', '0.48773291', '0.65576744']
['0.62749497', '0.23604815', '0.74252229', '0.64571741', '0.50962638', '0.67979353']
['0.61271797', '0.23130363', '0.65540763', '0.55365775', '0.50693777', '0.71493697']
['0.63108252', '0.23110159', '0.71736294', '0.57999278', '0.49490782', '0.65974977']


(array([0.00642651, 0.02803845, 0.06577803, 0.03542127, 0.00797837,
        0.02250737]),
 array([0.6223 , 0.22266, 0.67032, 0.60274, 0.50004, 0.68168]),
 0.02459198243330538,
 0.53548,
 array([0.5541, 0.4916, 0.5627, 0.5324, 0.5366]))

### pretrain

In [147]:
exp_type = 'OOD'
model_arch = 'hrnet'
pretrain_type = 'pretrain_v1'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.52043724', '0.20379863', '0.58378115', '0.51966846', '0.46494532', '0.59913198']
['0.60783599', '0.22055801', '0.68182263', '0.61414279', '0.56161366', '0.66641493']
['0.61655553', '0.21332656', '0.77071302', '0.48900341', '0.51349025', '0.62418342']
['0.61031695', '0.24293038', '0.60316506', '0.59470416', '0.48357488', '0.57884435']
['0.62056051', '0.22405848', '0.70227067', '0.56002246', '0.52397278', '0.66405555']


(array([0.03764283, 0.01299609, 0.06812085, 0.04624876, 0.03346828,
        0.03474797]),
 array([0.59514, 0.22094, 0.66836, 0.5555 , 0.50952, 0.62652]),
 0.026278325669646472,
 0.51616,
 array([0.4743, 0.5489, 0.5221, 0.5006, 0.5349]))

## SwinTransformer

In [133]:
exp_type = 'OOD'
model_arch = 'swin'
pretrain_type = 'normal'
transformer_flag = True

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path, transformer=transformer_flag)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.61132667', '0.14258206', '0.54877069', '0.55563071', '0.33149472', '0.52739387']
['0.60830508', '0.14691602', '0.64068669', '0.50802023', '0.36707527', '0.51763624']
['0.57887904', '0.14624864', '0.6341014', '0.48196259', '0.33966583', '0.52917972']
['0.60984567', '0.13287962', '0.63919419', '0.543981', '0.34050366', '0.53162702']
['0.61314033', '0.14582301', '0.65892362', '0.45996194', '0.37709814', '0.43717095']


(array([0.01278943, 0.00520361, 0.03868956, 0.03612016, 0.01765473,
        0.036016  ]),
 array([0.60428, 0.14288, 0.62434, 0.50992, 0.35118, 0.5086 ]),
 0.0084162699576475,
 0.42738000000000004,
 array([0.4212, 0.4361, 0.4262, 0.4376, 0.4158]))

In [134]:
exp_type = 'OOD'
model_arch = 'swin'
pretrain_type = 'pretrain'
transformer_flag = True

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path, transformer=transformer_flag)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.49293845', '0.14167731', '0.65186331', '0.50938729', '0.35765987', '0.57573153']
['0.62243286', '0.13648972', '0.69328951', '0.50823026', '0.29661877', '0.55470243']
['0.62982745', '0.13373061', '0.56212466', '0.53130403', '0.26464616', '0.48837576']
['0.62623272', '0.16219264', '0.62529072', '0.48858526', '0.40205575', '0.50285715']
['0.61953514', '0.14595097', '0.60281608', '0.51354431', '0.45789623', '0.48264787']


(array([0.05274461, 0.01002864, 0.04430848, 0.01361837, 0.0698481 ,
        0.03739682]),
 array([0.59816, 0.14402, 0.62708, 0.5102 , 0.35578, 0.52086]),
 0.018197252539875343,
 0.4316,
 array([0.4473, 0.4379, 0.396 , 0.4362, 0.4406]))

## FPN

### normal

In [58]:
exp_type = 'OOD'
model_arch = 'fpn'
pretrain_type = 'normal'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.62954768', '0.29855083', '0.67662297', '0.74733234', '0.59580098', '0.62274937']
['0.62883339', '0.29755921', '0.72270946', '0.69349483', '0.54014797', '0.72112256']
['0.64067639', '0.30063348', '0.74535834', '0.70767314', '0.55790674', '0.7022569']
['0.6416565', '0.30110436', '0.76458663', '0.77916369', '0.61888528', '0.6832765']
['0.63376147', '0.26390879', '0.71205162', '0.70887471', '0.50661121', '0.75963215']


(array([0.00543065, 0.01428742, 0.02998662, 0.03149212, 0.03982043,
        0.04523857]),
 array([0.6349 , 0.29236, 0.72428, 0.72732, 0.56386, 0.6978 ]),
 0.015008051172620652,
 0.60112,
 array([0.5882, 0.595 , 0.6028, 0.6294, 0.5902]))

### pretrain

In [59]:
exp_type = 'OOD'
model_arch = 'fpn'
pretrain_type = 'pretrain'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.52358687', '0.31372421', '0.72777046', '0.72624969', '0.57877014', '0.71613424']
['0.63751103', '0.33372787', '0.73064706', '0.74937176', '0.61951818', '0.6511651']
['0.62564714', '0.31470908', '0.77557727', '0.71232723', '0.51810673', '0.62486685']
['0.63449565', '0.32261323', '0.77745115', '0.6353827', '0.58141404', '0.54882806']
['0.63225102', '0.31641398', '0.80292921', '0.64781956', '0.56323942', '0.6116823']


(array([0.04372564, 0.0074155 , 0.0291539 , 0.04474159, 0.03278201,
        0.05443442]),
 array([0.6107 , 0.32022, 0.76288, 0.69422, 0.5722 , 0.63054]),
 0.016363618181808075,
 0.596,
 array([0.6125, 0.6169, 0.5891, 0.5731, 0.5884]))

## DeepLab

### normal

In [ ]:
exp_type = 'OOD'
model_arch = 'deeplab'
pretrain_type = 'normal'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

mious.std(axis=0), mious.mean(axis=0)

### pretrain

In [124]:
exp_type = 'OOD'
model_arch = 'deeplab'
pretrain_type = 'pretrain'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.42932088', '0.30356582', '0.6699681', '0.73564828', '0.53586319', '0.68126794']
['0.57604361', '0.34278708', '0.73094049', '0.80300012', '0.62506904', '0.77346522']
['0.62823588', '0.34777274', '0.76172969', '0.76794641', '0.60055597', '0.82936717']
['0.60415748', '0.30568595', '0.77461527', '0.70813859', '0.53810343', '0.76015423']
['0.43850537', '0.31382496', '0.70923503', '0.7566771', '0.57189286', '0.72257842']


(array([0.08442788, 0.01879911, 0.03748308, 0.03176958, 0.03482266,
        0.04974595]),
 array([0.53524, 0.32274, 0.72928, 0.75426, 0.57432, 0.7534 ]),
 0.028147753018669168,
 0.6268,
 array([0.5853, 0.6551, 0.6615, 0.6173, 0.6148]))

## Magnet

### normal

In [7]:
exp_type = 'OOD'
model_arch = 'magnet'
pretrain_type = 'normal'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.28563848', '0.35327155', '0.58523367', '0.68547363', '0.56915348', '0.72235648']
['0.50139802', '0.28099578', '0.57905077', '0.74105751', '0.4888026', '0.40892816']
['0.51262372', '0.31984798', '0.69079035', '0.83011051', '0.56658507', '0.59017629']
['0.4242108', '0.32367626', '0.61369646', '0.73855207', '0.56160921', '0.57629859']
['0.59575767', '0.30491956', '0.74793149', '0.70474431', '0.52784112', '0.7525469']


(array([0.10444379, 0.02371401, 0.06568228, 0.04967398, 0.0308475 ,
        0.12240011]),
 array([0.46392, 0.31654, 0.64334, 0.74   , 0.5428 , 0.61006]),
 0.03855886927802733,
 0.57056,
 array([0.5831, 0.4998, 0.5995, 0.5628, 0.6076]))

### pretrain

In [8]:
exp_type = 'OOD'
model_arch = 'magnet'
pretrain_type = 'pretrain'

# read one csv file of each IOU
base_path = f'/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/{model_arch}/{exp_type}/{pretrain_type}'

ious, mious = list_iou_model(base_path)

ious.std(axis=0), ious.mean(axis=0), mious.std(axis=0), mious.mean(axis=0), mious

['0.49573113', '0.17356584', '0.4871978', '0.33099083', '0.30802362', '0.37433639']
['0.2245953', '0.20151972', '0.49640961', '0.71492216', '0.40468541', '0.36812138']
['0.57610835', '0.22294357', '0.60061269', '0.51771959', '0.42193204', '0.40911559']
['0.554828', '0.22950438', '0.49486556', '0.63402187', '0.38765469', '0.44365245']
['0.57610835', '0.22294357', '0.60061269', '0.51771959', '0.42193204', '0.40911559']


(array([0.13371017, 0.02054385, 0.05288688, 0.12965706, 0.04236478,
        0.02738508]),
 array([0.48546, 0.21008, 0.53594, 0.54306, 0.38884, 0.40086]),
 0.040503017171563904,
 0.41575999999999996,
 array([0.3348, 0.4371, 0.4345, 0.4379, 0.4345]))

# Get Gaula vs Orkla

In [20]:
# for models
base_path = '/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/swin/OOD/normal/2023-05-03_01:13:41.811504_swin_transformer_freeze_0'
base_path = '/home/saeid/phd/segmentation/dataset/neurips dataset/2023 submission/results/'

all_model_paths = glob.glob( os.path.join(base_path, '*') )

overall_list = []

for model_path in all_model_paths:
    model_name = os.path.split(model_path)[1]
#     if model_name != 'unet':
#         continue
        
    print(f'\n Model {model_name}')
    
    # get all the experimetns
    all_normal_exp_paths =  glob.glob( os.path.join(model_path, 'OOD', 'normal', '*') )
    all_pretrain_exp_paths =  glob.glob( os.path.join(model_path, 'OOD', 'pretrain', '*') )
    
    all_normal_exp_paths.sort()
    all_pretrain_exp_paths.sort()
    
    gaula_model_normal = []
    gaula_model_pretrain = []
    orkla_model_normal = []
    orkla_model_pretrain = []
    
    for normal_exp in all_normal_exp_paths:
        # get all the gaulas and show
        gaula_path = glob.glob( os.path.join(normal_exp, '*_gaula_1998.txt') )[0]
        gaula_miou_n = read_miou_file(file_path=gaula_path)
        
        orkla_path = glob.glob( os.path.join(normal_exp, '*_orkla_1962.txt') )[0]
        orkla_miou_n = read_miou_file(file_path=orkla_path)
#         print(f' Normal gaula is {gaula_miou_n} orkla is {orkla_miou_n}')
        
        gaula_model_normal.append(gaula_miou_n)
        orkla_model_normal.append(orkla_miou_n)
        
    
    for pretrian_exp in all_pretrain_exp_paths:
        # get all the gaulas and show
        gaula_path = glob.glob( os.path.join(pretrian_exp, '*_gaula_1998.txt') )[0]
        gaula_miou_p = read_miou_file(file_path=gaula_path)
        
        orkla_path = glob.glob( os.path.join(pretrian_exp, '*_orkla_1962.txt') )[0]
        orkla_miou_p = read_miou_file(file_path=orkla_path)
#         print(f' Pretrained gaula is {gaula_miou_p} orkla is {orkla_miou_p}')
        
        gaula_model_pretrain.append(gaula_miou_p)
        orkla_model_pretrain.append(orkla_miou_p)
    
    # make the final list [ [name, gaula_model_normal, gaula_model_pretrain, orkla_model_normal, orkla_model_pretrain] ...]
    overall_list.append([model_name, gaula_model_normal, gaula_model_pretrain, orkla_model_normal, orkla_model_pretrain, 
                                    np.array(gaula_model_normal).mean(), np.array(gaula_model_pretrain).mean(),
                                     np.array(gaula_model_normal).std(), np.array(gaula_model_pretrain).std(),
                                    np.array(orkla_model_normal).mean(), np.array(orkla_model_pretrain).mean(),
                                     np.array(orkla_model_normal).std(), np.array(orkla_model_pretrain).std(),
                        ])
    
    # get all the orkla and show


 Model fpn

 Model unet

 Model deeplab

 Model swin

 Model hrnet


In [7]:
overall_list

[['fpn',
  [0.5836, 0.5434, 0.5647, 0.6295, 0.5723],
  [0.5794, 0.581, 0.5401, 0.5118, 0.5369],
  [0.5974, 0.6983, 0.6789, 0.6292, 0.626],
  [0.6788, 0.6887, 0.6872, 0.6959, 0.6915],
  0.5787000000000001,
  0.54984,
  0.64596,
  0.68842],
 ['unet',
  [0.6146, 0.6022, 0.6138, 0.6163, 0.6321],
  [0.5583, 0.5853, 0.5592, 0.5534, 0.5458],
  [0.6761, 0.7377, 0.6944, 0.6624, 0.6971],
  [0.6931, 0.6781, 0.6975, 0.6515, 0.6901],
  0.6157999999999999,
  0.5604,
  0.6935399999999999,
  0.68206],
 ['deeplab',
  [0.6395, 0.5707, 0.5639, 0.6078, 0.6384],
  [0.5354, 0.6364, 0.6319, 0.5822, 0.5859],
  [0.6687, 0.6814, 0.6871, 0.6355, 0.7391],
  [0.6851, 0.6924, 0.7206, 0.6876, 0.6728],
  0.6040599999999999,
  0.5943600000000001,
  0.68236,
  0.6917000000000001],
 ['swin',
  [0.4217, 0.4353, 0.4267, 0.425, 0.3951],
  [0.4424, 0.4332, 0.383, 0.4207, 0.4328],
  [0.4201, 0.4377, 0.4253, 0.4628, 0.4572],
  [0.457, 0.4471, 0.422, 0.4671, 0.456],
  0.42076,
  0.42241999999999996,
  0.4406199999999999,
  0.4